In [1]:
from sklearn.model_selection import train_test_split
from pathlib import Path
import matplotlib.image as img
import json
import numpy as np
from tqdm import tqdm
import h5py

from multiprocessing import Pool
import time
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

In [2]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [1]:
(100,200)[0]

100

In [3]:
# Helper Functions from EDA
def ejson(p, fn): # extract json
    with open((p/fn).as_posix()) as f: return json.load(f) 
## Simple Json Reading Functions
def info_data(p): return ejson(p, 'info.json')
def dot_data(p): return ejson(p, 'dotInfo.json')
def frame_data(p): return ejson(p, 'frames.json')    
def screen_data(p): return ejson(p, 'screen.json')
def get_frame(p, img_fn): return img.imread(p/'..'/'..'/'gazecapture-224x224'/p.name/'frames'/img_fn)
## Larger Helper Functions
def coordinate_data(p): 
    data = dot_data(p)
    return data['XCam'], data['YCam'] # we want relative to camera coords
def orientation_data(p):
    sdata = screen_data(p)
    return sdata['Orientation']

In [4]:
# dset_path = Path('/media/brennan/Data/ml/gazecapture/')
dset_path = Path('../gazecapture/')

# Export to hdf5
Not Used - See below for better soln.

In [9]:
# Extracting and storing X values
def extract_photo(f, case): return get_frame(case, f)

def extract(case):
    F = frame_data(case) # list of file names correlated to camera images 
    with Pool(6) as p:  # multi core usale
        x = p.map(functools.partial(extract_photo, case=case), F)
    return x

def extract_to_hdf5(cname, bs, extract_fcn): # bs dsets per hp5y file
    f = h5py.File('{}-0.hdf5'.format(cname), 'w') 
    for i, case in enumerate(list(dset_path.iterdir())):
        cn = case.name
        edata = extract_fcn(case)
        
        dset = f.create_dataset('dset'+cn, (len(edata), 224, 224, 3))
        dset[...] = edata

        if i % bs == 0 and i!= 0:
            f.close()
            f = h5py.File('{}-{}.hdf5'.format(cname, i//bs), 'w')

In [2]:
extract_to_hdf5('X', 500, extract)

# Extract File Names For Keras Generator

In [14]:
def extract_data(cases, accepted_o=[1, 2, 3, 4]):
    fnames = []
    XCam = []
    YCam = []
    
    i = 0
    for case in cases:
        F = frame_data(case)
        O = orientation_data(case)
        XCAM, YCAM = coordinate_data(case)

        for f, o, xcam, ycam in zip(F, O, XCAM, YCAM):
            if o in accepted_o:
                fnames.append('{}/frames/{}'.format(case.name, f))
                XCam.append(xcam)
                YCam.append(ycam)
            
    return fnames, XCam, YCam

In [22]:
test_size = 0.1

In [23]:
# Extract all case names first to split => faces it hasnt seen before
train_cases, test_cases = train_test_split(list(dset_path.iterdir()), test_size=test_size)

In [24]:
len(train_cases), len(test_cases)

(1326, 148)

In [18]:
# ALL PHOTOS split into train / test sets
fnames_test, XCam_test, YCam_test = extract_data(test_cases)
fnames_train, XCam_train, YCam_train = extract_data(train_cases)

In [10]:
# create dataframe from data
# HERE
train_df = pd.DataFrame(data={'file_names': fnames_train, 'XCam': XCam_train,
                       'YCam': YCam_train})
test_df = pd.DataFrame(data={'file_names': fnames_test, 'XCam': XCam_test,
                       'YCam': YCam_test})

In [12]:
train_df.to_csv('train-df.csv')
test_df.to_csv('test-df.csv')

In [25]:
# ONLY PORTRAIT photos in tests / train
fnames_test, XCam_test, YCam_test = extract_data(test_cases, accepted_o=[1])
fnames_train, XCam_train, YCam_train = extract_data(train_cases, accepted_o=[1])

train_df = pd.DataFrame(data={'file_names': fnames_train, 'XCam': XCam_train,
                       'YCam': YCam_train})
test_df = pd.DataFrame(data={'file_names': fnames_test, 'XCam': XCam_test,
                       'YCam': YCam_test})

train_df.to_csv('portrait-train-df.csv')
test_df.to_csv('portrait-test-df.csv')

In [26]:
len(test_df) / (len(test_df) + len(train_df)), len(train_df) / (len(test_df) + len(train_df))

(0.09589250062238537, 0.9041074993776146)

In [27]:
len(test_df), len(train_df)

(78577, 740851)

In [1]:
train_df_sample = train_df.sample(fac=0.2)
strain, sval = train_test_split(train_df_sample, test_size=0.1)
strain.to_csv('traindf_sample.csv'), sval.to_csv('valdf_sample.csv')

NameError: name 'train_df' is not defined